In [1]:
from settings.common import get_pp_pipeline, load_flat_dataset, save_flat_dataset
from settings.ngrams import insert_ngrams_flat_from_lists
from preprocessing_pipeline.NextGen import NextGen
import csv
from settings.common import load_flat_dataset
from settings.ngrams import insert_ngrams_flat_from_lists
import pandas as pd 
import os 
import gensim

In [2]:
dataset_name = 'fact_checker_data'
seed_topics_file = "data_gtm/{}_seed_topics.csv".format(dataset_name)
data_path = 'data_gtm/{}.csv'

pp = get_pp_pipeline(remove_stopwords=True, stem=False, clean_twitter=False)

ds_path = data_path.format(dataset_name+"_raw")

In [3]:
raw_data = pd.DataFrame(pd.read_csv(ds_path).claim)

In [4]:
raw_data.to_csv(data_path.format(dataset_name), header=False, index=False)

In [5]:
ds_path = data_path.format(dataset_name)

In [6]:
dataset = load_flat_dataset(ds_path, delimiter=' ')

In [7]:
dataset = []
from src.gdtm.helpers.common import load_flat_dataset
from src.gdtm.models import GTM, NLDA

tnd_path = '../topic-noise-models-source/mallet-tnd/bin/mallet'
lda_path = '../topic-noise-models-source/mallet-lda/bin/mallet'
gtm_path = '../topic-noise-models-source/mallet-gtm/bin/mallet'
dataset = load_flat_dataset("data_gtm/{}.csv".format(dataset_name))

In [8]:
clean_dataset = []
for data in dataset:
    tokens = gensim.utils.simple_preprocess(" ".join(data))
    clean_dataset.append(tokens)
dataset = clean_dataset

In [9]:
# remove all mentions in the data
dataset1 = []
for d in dataset:
    tmp = []
    for i in d:
        if '@' not in i:
            tmp.append(i)
    dataset1.append(tmp)
dataset = dataset1

In [10]:
# add bigrams to the dataset
ngrams = []
with open("data_gtm/{}_seed_topics.csv".format(dataset_name), 'r') as f:
    for line in f:
        topic = line.strip().split(',')
        for w in topic:
            if '$' in w:
                ngrams.append(w)
dataset = insert_ngrams_flat_from_lists(dataset, ngrams, [])
all_bigram = []
for i in dataset:
    tmp = []
    for index in range(0,len(i)-1):
        tmp.append('$'.join(i[index:index+2]))
    all_bigram.append(tmp)
dataset.extend(all_bigram)

In [11]:
model = GTM(dataset=dataset, mallet_tnd_path=tnd_path, mallet_gtm_path=gtm_path, 
              tnd_k=20, gtm_k=20, tnd_beta1 = 16, over_sampling_factor=100, phi=10,
              top_words=20,tnd_iterations=1000, gtm_iterations=1000, seed_topics_file=seed_topics_file)

In [12]:
topics = model.get_topics()
noise = model.get_noise_distribution()
topic_formula = model.show_topics()
coherence = model.get_coherent()
print("Noise:",noise)
print("\nTopics:", topic_formula)
print("\nUMass Coherence Score:", coherence)

Noise: [('president', 161), ('biden', 122), ('election', 103), ('joe', 91), ('joe$biden', 87), ('trump', 78), ('president$joe', 53), ('in$the', 52), ('of$the', 47), ('donald', 45), ('donald$trump', 45), ('ballots', 41), ('vote', 35), ('president$donald', 35), ('presidential', 33), ('former$president', 30), ('people', 26), ('the$election', 24), ('house', 23), ('shows', 23), ('georgia', 22), ('voting', 22), ('the$presidential', 22), ('presidential$election', 22), ('mail', 20), ('white', 20), ('million', 20), ('arizona', 19), ('to$vote', 19), ('state', 18), ('voter', 17), ('ukraine', 17), ('won', 16), ('hunter$biden', 16), ('states', 16), ('to$the', 16), ('on$the', 15), ('jobs', 15), ('covid', 14), ('illegal', 14), ('mail$in', 14), ('left', 13), ('republican', 13), ('year', 13), ('americans', 13), ('voters', 13), ('white$house', 13), ('created', 12), ('video$shows', 12), ('administration', 12), ('to$be', 12), ('show', 12), ('the$president', 12), ('passed', 11), ('from$the', 11), ('federal

In [13]:
noise_word = [term for term, wt in noise[:50]]
noise_wt = [wt for term, wt in noise[:50]]
noise_data = pd.DataFrame([noise_word, noise_wt]).T.rename(columns={0:"Noise_Word", 1:"Noise_Weight"}).sort_values(by="Noise_Weight", ascending=True)

In [14]:
clean_topic = []
for topic in topics:
    temp=[]
    for t in topic:
        if t not in noise_word:
            temp.append(t)
    clean_topic.append(temp)

In [15]:
with open('data_gtm/'+dataset_name+"_gtm_topics.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(clean_topic)

In [ ]:
cmd = 'rm /tmp/750b80*'
os.system(cmd)
cmd = 'rm /tmp/f86f5*'
os.system(cmd)